In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
np.random.seed(123)  # for reproducibility

from keras.models import Sequential, load_model, model_from_json, Model
from keras.applications.resnet50 import preprocess_input, ResNet50Net50
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import MaxPooling2D, Conv2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import SGD
from keras.metrics import categorical_accuracy

import tensorflow as tf
# run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

In [2]:
CLASS_MAP = {'antelope': 0,
 'bat': 1,
 'beaver': 2,
 'bobcat': 3,
 'buffalo': 4,
 'chihuahua': 5,
 'chimpanzee': 6,
 'collie': 7,
 'dalmatian': 8,
 'german+shepherd': 9,
 'grizzly+bear': 10,
 'hippopotamus': 11,
 'horse': 12,
 'killer+whale': 13,
 'mole': 14,
 'moose': 15,
 'mouse': 16,
 'otter': 17,
 'ox': 18,
 'persian+cat': 19,
 'raccoon': 20,
 'rat': 21,
 'rhinoceros': 22,
 'seal': 23,
 'siamese+cat': 24,
 'spider+monkey': 25,
 'squirrel': 26,
 'walrus': 27,
 'weasel': 28,
 'wolf': 29}
CLASS_WEIGHTS = [0.6235012 , 1.69270833, 3.25814536, 1.03668262, 0.71507151,
       1.12262522, 0.90845563, 0.6372549 , 1.20705664, 0.63076177,
       0.74328188, 0.93390805, 0.390039  , 2.24525043, 7.22222222,
       0.91036415, 3.49462366, 0.83493899, 0.86493679, 0.88255261,
       1.25240848, 1.96969697, 0.90845563, 0.65162907, 1.27077224,
       2.29276896, 0.53630363, 2.92792793, 2.35507246, 1.07526882]

In [3]:
batch_size = 16
resol = 224

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.5,
        zoom_range=0.1,
        horizontal_flip=True,
        preprocessing_function=preprocess_input,
        )
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(horizontal_flip=True,
                                  preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        'train_data/',  # this is the target directory
        target_size=(resol, resol),
        batch_size=batch_size,
        class_mode='categorical') 


# # this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'test_data/',
        target_size=(resol, resol),
        batch_size=batch_size,
        class_mode='categorical')

Found 10412 images belonging to 30 classes.
Found 2588 images belonging to 30 classes.


In [4]:
def load_top_model():
    model = Sequential()
    model.add(Dense(1024, activation='relu', input_shape=(2048,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(30, activation='softmax'))
    return model

In [5]:
base_model = ResNet50(include_top=False, pooling='avg')
top = load_top_model()

x = top(base_model.outputs)
model = Model(inputs=base_model.inputs, outputs=x)

In [6]:
for layer in base_model.layers:
    layer.trainable = False

In [7]:
lr_list = [0.0003, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001]
epoch_list = [30, 10, 10, 10, 10, 10]
for rnd in range(6):
    n_layers = rnd * 5
    if n_layers > 0:
        for layer in base_model.layers[-n_layers:]:
            layer.trainable = True
    n_trainable = sum(m.trainable for m in base_model.layers)
    print(f"\nFinetuning {n_trainable} hidden layers")
    
    opt = SGD(lr=lr_list[rnd], decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=[categorical_accuracy])
    
    path = f"checkpoints/ResNet/weights_unfreezed_{n_trainable}."
    model_checkpointer = ModelCheckpoint(filepath=path+"{epoch:02d}-{val_loss:.2f}.hdf5", 
                                        verbose=1, save_best_only=True, save_weights_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                                  patience=2, min_lr=0.0000001, verbose=1)
    early_stops = EarlyStopping(monitor='val_loss',
                    patience=15, verbose=1,)
    
    my_callbacks = []
    my_callbacks.append(model_checkpointer)
    my_callbacks.append(reduce_lr)
    my_callbacks.append(early_stops)
    
    history = model.fit_generator(
            train_generator,
            steps_per_epoch=10412 // batch_size, # 10412
            epochs=epoch_list[rnd],
            verbose=1,
            callbacks=my_callbacks,
            validation_data=validation_generator,
            validation_steps=2588 // batch_size, # 2588
            class_weight=CLASS_WEIGHTS,
            )
    pd.DataFrame(history.history).to_csv(f"RESULTS/ResNet/history_unfreezed_{n_trainable}.csv")
    model.save(f'MODEL/ResNet/densenet_unfreezed_{n_trainable}.h5') 
    


Finetuning 0 hidden layers
Epoch 1/30
 58/650 [=>............................] - ETA: 21:42 - loss: 3.6314 - categorical_accuracy: 0.1045

KeyboardInterrupt: 